In [17]:
# importing necessary libraries
import pandas as pd
import warnings
import requests
import tweepy
import geocoder
warnings.filterwarnings('ignore')
import seaborn as sns
from sklearn.naive_bayes import MultinomialNB
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split 
# from sklearn.preprocessing import  MinMaxScaler 
from sklearn.preprocessing import  StandardScaler 
from sklearn import metrics as mt
from spellchecker import SpellChecker 
import nltk
nltk.download('punkt')
from nltk import word_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk import FreqDist
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize, pos_tag
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')
from sklearn.model_selection import GridSearchCV
import re
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.datasets import make_classification

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [2]:
!pip install tweepy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install geocoder

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 98 kB 6.8 MB/s 


In [4]:
! pip install pyspellchecker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.5 MB 35.5 MB/s 


In [5]:
data = pd.read_csv("labeled_data.csv", on_bad_lines='skip')
data.head()

Unnamed: 0  count  hate_speech  offensive_language  neither  class  \
0           0      3            0                   0        3      2   
1           1      3            0                   3        0      1   
2           2      3            0                   3        0      1   
3           3      3            0                   2        1      1   
4           4      6            0                   6        0      1   

                                               tweet  
0  !!! RT @mayasolovely: As a woman you shouldn't...  
1  !!!!! RT @mleew17: boy dats cold...tyga dwn ba...  
2  !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...  
3  !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...  
4  !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...

In [6]:
data.shape

(24783, 7)

In [7]:
data.drop(data.columns[[0,1,2,3,4]], axis=1, inplace=True)
data.head()

class                                              tweet
0      2  !!! RT @mayasolovely: As a woman you shouldn't...
1      1  !!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2      1  !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3      1  !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4      1  !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...

In [8]:
data = data[data['class'] != 0]

In [9]:
data.shape

(23353, 2)

In [ ]:
data.loc[data['class']>1,'label'] = 0
data.loc[data['class']<=1,'label'] = 1
data.drop(data.columns[[0]], axis=1, inplace=True)
data.head()

In [11]:
data['tweet'] = data['tweet'].str.lower()
data.head()

<ipython-input-11-cb5f39728210>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['tweet'] = data['tweet'].str.lower()


tweet  label
0  !!! rt @mayasolovely: as a woman you shouldn't...    0.0
1  !!!!! rt @mleew17: boy dats cold...tyga dwn ba...    1.0
2  !!!!!!! rt @urkindofbrand dawg!!!! rt @80sbaby...    1.0
3  !!!!!!!!! rt @c_g_anderson: @viva_based she lo...    1.0
4  !!!!!!!!!!!!! rt @shenikaroberts: the shit you...    1.0

In [12]:
data.isna().sum()

tweet    0
label    0
dtype: int64

In [13]:
def removeSpace(tweet):
  return " ".join(tweet.split())

In [14]:
data['tweet'] = data['tweet'].apply(removeSpace)

In [15]:
data.head()

tweet  label
0  !!! rt @mayasolovely: as a woman you shouldn't...    0.0
1  !!!!! rt @mleew17: boy dats cold...tyga dwn ba...    1.0
2  !!!!!!! rt @urkindofbrand dawg!!!! rt @80sbaby...    1.0
3  !!!!!!!!! rt @c_g_anderson: @viva_based she lo...    1.0
4  !!!!!!!!!!!!! rt @shenikaroberts: the shit you...    1.0

In [18]:
data['tweet'] = data['tweet'].apply(lambda x: word_tokenize(x))

In [19]:
data.head()

tweet  label
0  [!, !, !, rt, @, mayasolovely, :, as, a, woman...    0.0
1  [!, !, !, !, !, rt, @, mleew17, :, boy, dats, ...    1.0
2  [!, !, !, !, !, !, !, rt, @, urkindofbrand, da...    1.0
3  [!, !, !, !, !, !, !, !, !, rt, @, c_g_anderso...    1.0
4  [!, !, !, !, !, !, !, !, !, !, !, !, !, rt, @,...    1.0

In [20]:
# def rectifySpelling(tweet):
#   sp = SpellChecker()
#   output = []
#   for line in tweet:
#     output.append(sp.correction(line))
#   return output

In [21]:
# data['tweet'] = data['tweet'].apply(rectifySpelling)

In [22]:
data.head()

tweet  label
0  [!, !, !, rt, @, mayasolovely, :, as, a, woman...    0.0
1  [!, !, !, !, !, rt, @, mleew17, :, boy, dats, ...    1.0
2  [!, !, !, !, !, !, !, rt, @, urkindofbrand, da...    1.0
3  [!, !, !, !, !, !, !, !, !, rt, @, c_g_anderso...    1.0
4  [!, !, !, !, !, !, !, !, !, !, !, !, !, rt, @,...    1.0

In [23]:
stopwords = stopwords.words('english')
#stop_words = set(stopwords.words("english"))
def removeWords(tweet):
  
  output = []
  for word in tweet:
    if word not in stopwords:
      output.append(word)
  return output


In [24]:
data['tweet'] = data['tweet'].apply(removeWords)

In [25]:
data.head()

tweet  label
0  [!, !, !, rt, @, mayasolovely, :, woman, n't, ...    0.0
1  [!, !, !, !, !, rt, @, mleew17, :, boy, dats, ...    1.0
2  [!, !, !, !, !, !, !, rt, @, urkindofbrand, da...    1.0
3  [!, !, !, !, !, !, !, !, !, rt, @, c_g_anderso...    1.0
4  [!, !, !, !, !, !, !, !, !, !, !, !, !, rt, @,...    1.0

In [26]:
def removePunctuation(tweet):
  token = RegexpTokenizer(r"\w+")
  return token.tokenize(' '.join(tweet))

In [27]:
data['tweet'] = data['tweet'].apply(removePunctuation)

In [28]:
data.head()

tweet  label
0  [rt, mayasolovely, woman, n, t, complain, clea...    0.0
1  [rt, mleew17, boy, dats, cold, tyga, dwn, bad,...    1.0
2  [rt, urkindofbrand, dawg, rt, 80sbaby4life, ev...    1.0
3  [rt, c_g_anderson, viva_based, look, like, tra...    1.0
4  [rt, shenikaroberts, shit, hear, might, true, ...    1.0

In [29]:
def freqWords(tweet):
  lines = tweet.values
  output = []
  for line in lines:
    output+=line[0]
  return FreqDist(output).most_common(10)

In [30]:
wordFreq = freqWords(data)
wordFreq

[('bitch', 8139),
 ('rt', 7319),
 ('t', 6525),
 ('n', 3872),
 ('128514', 3145),
 ('bitches', 3060),
 ('co', 2884),
 ('http', 2798),
 ('like', 2618),
 ('s', 2378)]

In [31]:
frequencies = []
for word, freq in wordFreq:
  if word=='bitch' or word=='bitches':
    pass
  else:
    frequencies.append(freq)
frequencies

[7319, 6525, 3872, 3145, 2884, 2798, 2618, 2378]

In [32]:
def removeFreqWord(tweet):
  output = []
  for line in tweet:
    if line not in frequencies:
      output.append(line)
  return output
  

In [33]:
data['tweet'] = data['tweet'].apply(removeFreqWord)

In [34]:
data.head()

tweet  label
0  [rt, mayasolovely, woman, n, t, complain, clea...    0.0
1  [rt, mleew17, boy, dats, cold, tyga, dwn, bad,...    1.0
2  [rt, urkindofbrand, dawg, rt, 80sbaby4life, ev...    1.0
3  [rt, c_g_anderson, viva_based, look, like, tra...    1.0
4  [rt, shenikaroberts, shit, hear, might, true, ...    1.0

In [35]:
def lemmatize(tweet):
  output = []
  lst = ['a','r','n','v']
  wordNetLemmatizer = WordNetLemmatizer()
  posTag = pos_tag(tweet)
  for word, tag in posTag:
    pos = tag[0].lower()

    if pos not in lst:
      pos = 'n'
    output.append(wordNetLemmatizer.lemmatize(word,pos))
  return output

In [36]:
data['tweet'] = data['tweet'].apply(lemmatize)

In [37]:
data.head()

tweet  label
0  [rt, mayasolovely, woman, n, t, complain, clea...    0.0
1  [rt, mleew17, boy, dat, cold, tyga, dwn, bad, ...    1.0
2  [rt, urkindofbrand, dawg, rt, 80sbaby4life, ev...    1.0
3  [rt, c_g_anderson, viva_based, look, like, tra...    1.0
4  [rt, shenikaroberts, shit, hear, might, true, ...    1.0

In [38]:
def removeTag(tweet):
  tweet=' '.join(tweet)
  return re.compile('<.*?>').sub(r'',tweet)

In [39]:
data['tweet'] = data['tweet'].apply(removeTag)

In [40]:
# def removeURL(s):
#   df['result'] = df['result'].str.replace(r'\D', '')
#   return re.compile(r'https?://\S+|www\.\S+').sub(r'',tweet)

In [41]:
#data['tweet'] = data['tweet'].apply(removeURL)
data['tweet'] = data['tweet'].str.replace(r'http', '')

In [42]:
data['tweet'][0]

'rt mayasolovely woman n t complain cleaning house amp man always take trash'

In [43]:
x= data['tweet']
y = data['label']

In [44]:
print(x.shape)
print(y.shape)

(23353,)
(23353,)


In [45]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.3, random_state=True)
print(x_train.shape)
print(x_test.shape)

(16347,)
(7006,)


In [103]:
def metric_calculation(y_test,y_pred):
  print("Accuracy  :  ",mt.accuracy_score(y_test,y_pred))
  print("Precision :  ",mt.precision_score(y_test,y_pred))
  print("Recall    :  ",mt.recall_score(y_test,y_pred))
  print("F1-score  :  ",mt.f1_score(y_test,y_pred))
  print("Confusion matrix      : \n ",mt.confusion_matrix(y_test,y_pred))
  print("Classification report : \n",mt.classification_report(y_test,y_pred))

In [46]:
cv=CountVectorizer()
x_train=cv.fit_transform(x_train)
x_test = cv.transform(x_test)

In [47]:
print(x_train.shape)
print(x_test.shape)

(16347, 24859)
(7006, 24859)


In [48]:
y_train = y_train.astype('int')

In [101]:
nb = MultinomialNB()
nb.fit(x_train,y_train)
y_pred = nb.predict(x_test)
metric_calculation(y_test,y_pred)

Accuracy  :   0.9082215244076506
Precision :   0.9086816720257235
Recall    :   0.9869041382922996
F1-score  :   0.9461789570603499
Confusion matrix      : 
  [[ 711  568]
 [  75 5652]]
Classification report :                 precision    recall  f1-score   support

         0.0       0.90      0.56      0.69      1279
         1.0       0.91      0.99      0.95      5727

    accuracy                           0.91      7006
   macro avg       0.91      0.77      0.82      7006
weighted avg       0.91      0.91      0.90      7006



In [50]:
# # tuning parameters for svm
# from sklearn.svm import SVC
# cost=[2,6]
# deg = [2,4,5]
# gamma= [0.005,0.05,0.5,1]
# kernel = ['rbf','linear','poly']
# max_score=0
# for c in cost:
#   for d in deg:
#     for g in gamma:
#       svm=SVC(C=c, kernel='rbf', degree=d, gamma = g)
#       svm.fit(x_train,y_train)
#       print(c,d,g,svm.score(x_test,y_test))
#       if(svm.score(x_test,y_test)>=max_score):
#         max_score=svm.score(x_test,y_test)
#         max_c=c
#         max_d=d
#         max_g=g
# print("score:, cost, degree, gamma")    
# print(max_score,max_c,max_d,max_g)

In [51]:
svc=SVC()
svc.fit(x_train,y_train)
y_pred_svm=svc.predict(x_test)
metric_calculation(y_test,y_pred_svm)
# print("Accuracy:\n",mt.accuracy_score(y_test,y_pred_svm))
# print("Precision:\n",mt.precision_score(y_test,y_pred_svm))
# print("Recall:\n",mt.recall_score(y_test,y_pred_svm))
# print("F1-score:\n",mt.f1_score(y_test,y_pred_svm))
# print("Confusion matrix:\n",mt.confusion_matrix(y_test,y_pred_svm))
# print("Classification report:\n",mt.classification_report(y_test,y_pred_svm))

Accuracy:
 0.9628889523265772
Precision:
 0.9857828327705704
Recall:
 0.9685699319015191
F1-score:
 0.9771005812929364
Confusion matrix:
 [[1199   80]
 [ 180 5547]]
Classification report:
               precision    recall  f1-score   support

         0.0       0.87      0.94      0.90      1279
         1.0       0.99      0.97      0.98      5727

    accuracy                           0.96      7006
   macro avg       0.93      0.95      0.94      7006
weighted avg       0.96      0.96      0.96      7006



In [52]:
# max_features = ['auto', 'sqrt', 'log2']
# max_depth = [5,6,7,8]
# criterion = ['gini', 'entropy']
# dt = DecisionTreeClassifier(random_state=0)
# gs_dt = GridSearchCV(dt, param_grid = { 'max_features':max_features,'max_depth':max_depth,'criterion':criterion})
# gs_dt.fit(x_train,y_train)

In [53]:
# gs_dt.best_estimator_

In [55]:
dt_best= DecisionTreeClassifier(max_depth=8, max_features='auto', random_state=0)
dt_best.fit(x_train,y_train)
y_pred_dt=dt_best.predict(x_test)

metric_calculation(y_test,y_pred_dt)
# print("Accuracy:\n",mt.accuracy_score(y_test,y_pred_dt))
# print("Precision:\n",mt.precision_score(y_test,y_pred_dt))
# print("Recall:\n",mt.recall_score(y_test,y_pred_dt))
# print("F1-score:\n",mt.f1_score(y_test,y_pred_dt))
# print("Confusion matrix:\n",mt.confusion_matrix(y_test,y_pred_dt))
# print("Classification report:\n",mt.classification_report(y_test,y_pred_dt))

Accuracy:
 0.8214387667713389
Precision:
 0.8218291630716135
Recall:
 0.997904662126768
F1-score:
 0.9013484740951029
Confusion matrix:
 [[  40 1239]
 [  12 5715]]
Classification report:
               precision    recall  f1-score   support

         0.0       0.77      0.03      0.06      1279
         1.0       0.82      1.00      0.90      5727

    accuracy                           0.82      7006
   macro avg       0.80      0.51      0.48      7006
weighted avg       0.81      0.82      0.75      7006



In [56]:
# rf= RandomForestClassifier()
# n_estimators = [200, 500]
# max_features = ['auto', 'sqrt', 'log2']
# max_depth = [5,7]
# criterion = ['gini', 'entropy']
# gs = GridSearchCV(rf, param_grid = {'n_estimators':n_estimators, 'max_features':max_features,'max_depth':max_depth,'criterion':criterion})
# gs.fit(x_train,y_train)

In [57]:
# gs.best_estimator_

In [58]:
rf_best= RandomForestClassifier(max_depth=5, n_estimators=200, random_state=0)
rf_best.fit(x_train,y_train)
y_pred_rf=rf_best.predict(x_test)

metric_calculation(y_test,y_pred_rf)
# print("Accuracy:\n",mt.accuracy_score(y_test,y_pred_rf))
# print("Precision:\n",mt.precision_score(y_test,y_pred_rf))
# print("Recall:\n",mt.recall_score(y_test,y_pred_rf))
# print("F1-score:\n",mt.f1_score(y_test,y_pred_rf))
# print("Confusion matrix:\n",mt.confusion_matrix(y_test,y_pred_rf))
# print("Classification report:\n",mt.classification_report(y_test,y_pred_rf))

Accuracy:
 0.8174421924065087
Precision:
 0.8174421924065087
Recall:
 1.0
F1-score:
 0.8995523443022069
Confusion matrix:
 [[   0 1279]
 [   0 5727]]
Classification report:
               precision    recall  f1-score   support

         0.0       0.00      0.00      0.00      1279
         1.0       0.82      1.00      0.90      5727

    accuracy                           0.82      7006
   macro avg       0.41      0.50      0.45      7006
weighted avg       0.67      0.82      0.74      7006



In [59]:
grid={ "penalty":["l1","l2"]}# l1 lasso l2 ridge
logreg=LogisticRegression()
logreg_cv=GridSearchCV(logreg,grid,cv=10)
logreg_cv.fit(x_train,y_train)

GridSearchCV(cv=10, estimator=LogisticRegression(),
             param_grid={'penalty': ['l1', 'l2']})

In [60]:
logreg_cv.best_params_

{'penalty': 'l2'}

In [61]:
logreg_best= LogisticRegression(penalty= 'l2')
logreg_best.fit(x_train,y_train)
y_pred_logreg=logreg_best.predict(x_test)

metric_calculation(y_test,y_pred_logreg)
# print("Accuracy:\n",mt.accuracy_score(y_test,y_pred_logreg))
# print("Precision:\n",mt.precision_score(y_test,y_pred_logreg))
# print("Recall:\n",mt.recall_score(y_test,y_pred_logreg))
# print("F1-score:\n",mt.f1_score(y_test,y_pred_logreg))
# print("Confusion matrix:\n",mt.confusion_matrix(y_test,y_pred_logreg))
# print("Classification report:\n",mt.classification_report(y_test,y_pred_logreg))

Accuracy:
 0.9603197259491865
Precision:
 0.9811054211548649
Recall:
 0.9701414353064431
F1-score:
 0.9755926251097454
Confusion matrix:
 [[1172  107]
 [ 171 5556]]
Classification report:
               precision    recall  f1-score   support

         0.0       0.87      0.92      0.89      1279
         1.0       0.98      0.97      0.98      5727

    accuracy                           0.96      7006
   macro avg       0.93      0.94      0.93      7006
weighted avg       0.96      0.96      0.96      7006



In [62]:
vc=VotingClassifier(estimators=[('logreg_best',logreg_best),('svc',svc),('rf_best',rf_best)],voting='hard')
vc.fit(x_train,y_train)
y_pred_vc=vc.predict(x_test)

metric_calculation(y_test,y_pred_vc)
# print("Accuracy:\n",mt.accuracy_score(y_test,y_pred_vc))
# print("Precision:\n",mt.precision_score(y_test,y_pred_vc))
# print("Recall:\n",mt.recall_score(y_test,y_pred_vc))
# print("F1-score:\n",mt.f1_score(y_test,y_pred_vc))
# print("Confusion matrix:\n",mt.confusion_matrix(y_test,y_pred_vc))
# print("Classification report:\n",mt.classification_report(y_test,y_pred_vc))

Accuracy:
 0.9620325435341136
Precision:
 0.978615249780894
Recall:
 0.9748559455212154
F1-score:
 0.9767319804058783
Confusion matrix:
 [[1157  122]
 [ 144 5583]]
Classification report:
               precision    recall  f1-score   support

         0.0       0.89      0.90      0.90      1279
         1.0       0.98      0.97      0.98      5727

    accuracy                           0.96      7006
   macro avg       0.93      0.94      0.94      7006
weighted avg       0.96      0.96      0.96      7006



Twitter

In [63]:
consumer_key = 'HZRzp3wipiTD7mPuHb6TbayFO'
consumer_secret = 'vfsSVritLxSC7traklojC7knAi4qGDPzrFsbCsAzCqfEyeNXRH'
access_token = '1593233208648224768-CUfwIsUxxZhZD1mKzRvPIn7fSmar99'
access_token_secret = 'stK9CfoU21OEsF3T2hwO7DnelvxCpNWxhe1tq2CAB7ScJ'

In [64]:
auth=tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token,access_token_secret)
api=tweepy.API(auth)

In [65]:
# cursor = tweepy.Cursor(api.user_timeline,id='@Raha',tweet_mode="extended").items(1)
# cursor

In [66]:
l=[]
closest_loc = api.trends_closest(51.5072, 0.1276)
trends = api.trends_place(closest_loc[0]["woeid"])

for i in range(47): 
  s=trends[0]["trends"][i]["name"]
  if s[0]=='#':
    print(s)
    l.append(s)

#MORSPA
#PORSUI
#MARESP
#Duolingo365
#TheTraitors


In [67]:
closest_loc[0]

{'name': 'London',
 'placeType': {'code': 7, 'name': 'Town'},
 'url': 'http://where.yahooapis.com/v1/place/44418',
 'parentid': 23424975,
 'country': 'United Kingdom',
 'woeid': 44418,
 'countryCode': 'GB'}

In [68]:
k=[]
main=[]
for j in range(3):
    for tweet in tweepy.Cursor(api.search, q=l[j]).items(10):
        print(tweet.text)
        k.append(tweet.text)
        
        print("___________________")
  

RT @ScheffenC: Et certains croient encore a l'assimilation. Faites attention a vos voitures ce soir à #Nantes. #MORSPA https://t.co/0f9MSFY…
___________________
@FIFAWorldCup #FIFAWorldCup: Alhamdulillah! Allah Hu Akbar ☝#Morocco into the Quarter-Finals. Arab leaders sold Pal… https://t.co/AQ9vOxTGQW
___________________
RT @iihtishamm: The greatest 🙏 #MORSPA https://t.co/FM1u73RI2h
___________________
RT @emiyxh_: One of the happiest nights of my life.🥺🇲🇦💚
#TeamMorocco
#MORSPA https://t.co/vMEG1kVRrX
___________________
RT @thatguyreddevil: There are scenes then there is this. #MORSPA https://t.co/KqCVrkp7Ga
___________________
RT @MicallefPhilip: @Football__Tweet Morocco did nothing with no ball and Spain did nothing with plenty of ball. A classic stalemate for mi…
___________________
RT @UBET__Sports: This is the man of the match. 
#MORSPA 
#FIFAWorldCup 
#Qatar2022 https://t.co/BQhwJnSoxY
___________________
RT @BrightonBDS: #Morocco fans sharing their big win with #Palestine in cen

In [69]:
# k.append("Fuck you")

In [70]:
#dir(api)

In [71]:
df = pd.DataFrame(k)
df.head()

0
0  RT @ScheffenC: Et certains croient encore a l'...
1  @FIFAWorldCup #FIFAWorldCup: Alhamdulillah! Al...
2  RT @iihtishamm: The greatest 🙏 #MORSPA https:/...
3  RT @emiyxh_: One of the happiest nights of my ...
4  RT @thatguyreddevil: There are scenes then the...

In [72]:
df.columns=['Tweet']

In [73]:
df['Tweet'] = df['Tweet'].str.replace('RT', '1111', regex=True)
df['Tweet'] = df['Tweet'].str.replace(' ', '1111', regex=True)
df['Tweet'] = df['Tweet'].str.replace('\W', '', regex=True)
df['Tweet'] = df['Tweet'].str.replace('1111', ' ', regex=True)
res = [idx for idx in df['Tweet'] if not re.findall("[^\u0000-\u05C0\u2100-\u214F]+", idx)]

In [74]:
res=pd.DataFrame(res)

In [75]:
res.columns=['Tweet']
copy=res
copy.head()

Tweet
0    ScheffenC Et certains croient encore a lassi...
1  FIFAWorldCup FIFAWorldCup Alhamdulillah Allah ...
2    iihtishamm The greatest  MORSPA httpstcoFM1u...
3    emiyxh_ One of the happiest nights of my lif...
4    thatguyreddevil There are scenes then there ...

In [76]:
res['Tweet'] = res['Tweet'].str.lower()
res['Tweet'] = res['Tweet'].apply(removeSpace)
res['Tweet'] = res['Tweet'].apply(lambda x: word_tokenize(x))
# res['tweet'] = res['tweet'].apply(rectifySpelling)
res['Tweet'] = res['Tweet'].apply(removeWords)
res['Tweet'] = res['Tweet'].apply(removePunctuation)
wordFreq = freqWords(res)
res['Tweet'] = res['Tweet'].apply(removeFreqWord)
res['Tweet'] = res['Tweet'].apply(lemmatize)
res['Tweet'] = res['Tweet'].apply(removeTag)
res['Tweet'] = res['Tweet'].str.replace(r'http', '')

In [77]:
# a=res["Tweet"]
# for i in range(len(a)):
#   print(a)

In [78]:
a=res["Tweet"]
x=cv.fit_transform(x)
test_df = cv.transform(a)

In [79]:
#svc_t=SVC()
vc.fit(x,y)
y_pred_tvc = vc.predict(test_df)

In [80]:
print(x_train.shape)
print(y_train.shape)
print(test_df.shape)

(16347, 24859)
(16347,)
(30, 31651)


In [81]:
z=[]
for i in range(len(a)):
  z.append(a[i])  
  print(a[i],y_pred_tvc[i])

scheffenc et certains croient encore lassimilation faites attention vos voitures ce soir à nantes morspa stco0f9msfy 0.0
fifaworldcup fifaworldcup alhamdulillah allah hu akbar morocco quarterfinal arab leader sell pal stcoaq9voxtgqw 0.0
iihtishamm greatest morspa stcofm1u73ri2h 0.0
emiyxh_ one happiest night lifeteammoroccomorspa stcovmeg1kvrrx 0.0
thatguyreddevil scene morspa stcokqcvrkp7ga 0.0
micallefphilip football__tweet morocco nothing ball spain nothing plenty ball classic stalemate mi 1.0
ubet__sports man match morspa fifaworldcup qatar2022 stcobqhwjnsoxy 0.0
brightonbds morocco fan share big win palestine central london tonightmorspa worldcup2022 stcotxsjjwizbx 0.0
inuka modern range_ consist signature perfume product modern twist encapsulate youth vib stcobtw7x8xxpg 0.0
iihtishamm moroccan player hold palestine flag victory morspa morocco stconwpduacqww 0.0
drgauravgarg4 eye glue one person porsui portugal fifaworldcup cristiano ronaldo stcolyirrwqqlo 0.0
abdellahboulma rappe

In [82]:
df_z=pd.DataFrame(z)
df_y_pred=pd.DataFrame(y_pred_tvc)

In [83]:
result = pd.concat([copy['Tweet'], df_y_pred], axis=1)

In [84]:
result.columns=['Tweet','Label']
result.head()

Tweet  Label
0  scheffenc et certains croient encore lassimila...    0.0
1  fifaworldcup fifaworldcup alhamdulillah allah ...    0.0
2          iihtishamm greatest morspa stcofm1u73ri2h    0.0
3  emiyxh_ one happiest night lifeteammoroccomors...    0.0
4        thatguyreddevil scene morspa stcokqcvrkp7ga    0.0

In [85]:
print(result['Tweet'][11],result['Label'][11])

abdellahboulma rappelle que gonçalo ramos 2 1ans qui vient de doubler la mise en quart était ciblé par le psg lété dernier 0.0


In [86]:
# 
# a=["This is a sunny morning",
# "Shut the fuck up",
# "Bonjour",
# "You have a very ugly face.",
# "Dont be a jerk",
# "You asshole",
# "Snow is white",
# "Happy morning",
# "Astala vista"]
# #a=["RT ukiswitheu Goves car crash interview about Michelle Mones PPEToryCorruption httpstcoM5sH5pgwpT"]
# data = np.array(a)
 

# # a=pd.DataFrame(a)
# test_df = cv.transform(a)
# y_pred_t = vc.predict(test_df)

In [87]:
# y_pred_t

In [94]:
result['Tweet'][29]

'eure_d ce racailles sont notre enfer maresp stop migrant caf rsa darmanin macronistan'

In [89]:
result['Tweet'][5]

'micallefphilip football__tweet morocco nothing ball spain nothing plenty ball classic stalemate mi'

In [90]:
# import numpy as np
# b=["white"]
# data = np.array(b)
# test_df_b = cv.transform(b)
# y_pred_t = vc.predict(test_df_b)
# print(vc.predict(test_df_b))

In [91]:
result['Tweet'] = result['Tweet'].str.replace(r'http', '')

In [ ]:
result

In [108]:
# for tweet in tweepy.Cursor(api.search, q="#slang").items(10):
#   print(tweet.text)
#   k.append(tweet.text)